In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json, re, requests, urllib, time, os
from bs4 import BeautifulSoup
from data_collection import load_data
from data_collection import get_drive_data
from data_collection import get_hike_pages
from data_collection import main
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
%load_ext autoreload
%autoreload 2

In [2]:
hikes = pd.read_csv('wta-parks-data.csv', sep='\t')

In [3]:
hikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 29 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Unnamed: 0                                                 3412 non-null   int64  
 1   name                                                       3412 non-null   object 
 2   url                                                        3412 non-null   object 
 3   length                                                     1921 non-null   float64
 4   highest point                                              1492 non-null   float64
 5   gain                                                       1492 non-null   float64
 6   lat                                                        2273 non-null   float64
 7   lon                                                        2273 non-null   float64
 8   pass: Di

In [4]:
hikes.head(2)

,Unnamed: 0,name,url,length,highest point,gain,lat,lon,pass: Discover Pass,pass: National Monument Fee,...,Fall foliage,Coast,Mountain views,Wildlife,Old growth,Summits,Ridges/passes,Established campsites,Waterfalls,Rivers
0,0,Luther Burbank Park,http://www.wta.org/go-hiking/hikes/luther-burb...,3.0,120.0,120.0,47.5890,-122.2270,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Gem Lake,http://www.wta.org/go-hiking/hikes/gem-lake,10.0,4857.0,1800.0,47.4453,-121.4235,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
client = MongoClient('localhost', 27017)
db = client['wta']
collection = db['hike_pages']

In [105]:
soup = BeautifulSoup(collection.find()[0]['content'],'html')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns:xlink="http://www.w3.org/1999/xlink">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!--
        /* @license
         * MyFonts Webfont Build ID 2229066, 2012-04-12T15:53:09-0400
         *
         * The fonts listed in this notice are subject to the End User License
         * Agreement(s) entered into by the website owner. All other parties are
         * explicitly restricted from using the Licensed Webfonts(s).
         *
         * You may obtain a valid license at the URLs below.
         *
         * Webfont: URW Egyptienne Narrow Medium by URW++
         * URL: http://www.myfonts.com/fonts/urw/egyptienne/urw-egyptienne-t-medium-narrow/
         * Copyright: Copyright 2010 URW++ Design &amp; Development Hamburg
         * Licensed pageviews: 10,000,000
         *
         *
         * Lic

In [18]:
hikes.head()

,Unnamed: 0,name,url,length,highest point,gain,lat,lon,pass: Discover Pass,pass: National Monument Fee,...,Fall foliage,Coast,Mountain views,Wildlife,Old growth,Summits,Ridges/passes,Established campsites,Waterfalls,Rivers
0,0,Luther Burbank Park,http://www.wta.org/go-hiking/hikes/luther-burb...,3.0,120.0,120.0,47.5890,-122.2270,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Gem Lake,http://www.wta.org/go-hiking/hikes/gem-lake,10.0,4857.0,1800.0,47.4453,-121.4235,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Talapus and Olallie Lakes,http://www.wta.org/go-hiking/hikes/talapus-lake,6.2,3780.0,1220.0,47.4007,-121.5190,0,0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,Norse Peak,http://www.wta.org/go-hiking/hikes/norse-peak,10.0,6856.0,2900.0,46.9644,-121.4829,0,0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,Rachel Lake,http://www.wta.org/go-hiking/hikes/rachel-lake,8.0,4600.0,1600.0,47.4012,-121.2848,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0


In [34]:
location = soup.find('div', {'class': 'hike-stat'}).text.split('\n')[2]
location

'Puget Sound and Islands'

In [38]:
length_x = soup.find('div', {'id': 'distance'}).text
length_temp = float(re.findall(r"(\d+\.+\d)", length_x)[0])
length_mult = 2 if not length_x == None and 'one-way' in length_x else 1
length = length_temp*length_mult
length

3.0

In [62]:
gain = float(soup.find_all('div', {'class': 'hike-stat'})[2].find_all('span')[0].text)
gain

120.0

In [63]:
highest_point = float(soup.find_all('div', {'class': 'hike-stat'})[2].find_all('span')[1].text)
highest_point

120.0

In [84]:
rating = float(soup.find('div', {'class': 'current-rating'}).text.split()[0])
rating

3.0

In [85]:
votes = int(soup.find('div', {'class': 'rating-count'}).text.split()[0][1:])
votes

5

In [86]:
description = soup.find('div', {'id': 'hike-body-text'}).text
description

"\nLuther Burbank Park is a waterfront park at the North end of Mercer Island, offering three miles of family- and pet-friendly trails with waterfront access, wetlands, and recreational activities. The park covers 77 acres, also including a children's play area, restrooms, an off-leash dog area, tennis courts, fishing piers, beaches, earthwork sculpture, and picnic areas.The park grounds were originally home to the Calkins Hotel in the late nineteenth century. The area next became the site of a private school teaching farm work and study. It has also housed the administrative offices of the King County Park System. The City of Mercer Island purchased the site in 2003.The park has a mix of paved and unpaved trails, mostly in the north-south direction. From the south parking lot, head north along the trail, past The Source, a spiral earthwork sculpture by John Hoge. The pathways will take you past playgrounds and dog play areas to Calkins Point, at the north end of the Island. Near Calki

In [87]:
report_count = int(soup.find('span', {'class': 'ReportCount'}).text)
report_count

15

In [93]:
lat = float(soup.find('div', {'class': 'latlong'}).find_all('span')[0].text)
lon = float(soup.find('div', {'class': 'latlong'}).find_all('span')[1].text)
lat, lon

(47.589, -122.227)

In [107]:
from data_collection import merge_pages

hikes2 = merge_pages('wta-parks-data.csv','')

0 http://www.wta.org/go-hiking/hikes/luther-burbank-park
1 http://www.wta.org/go-hiking/hikes/gem-lake
2 http://www.wta.org/go-hiking/hikes/talapus-lake
3 http://www.wta.org/go-hiking/hikes/norse-peak
4 http://www.wta.org/go-hiking/hikes/rachel-lake
5 http://www.wta.org/go-hiking/hikes/blanca-lake
6 http://www.wta.org/go-hiking/hikes/bearhead
7 http://www.wta.org/go-hiking/hikes/colchuck-lake
8 http://www.wta.org/go-hiking/hikes/naches-peak-loop
9 http://www.wta.org/go-hiking/hikes/goat-peak
10 http://www.wta.org/go-hiking/hikes/rattle-snake-ledge
11 http://www.wta.org/go-hiking/hikes/tatoosh-lakes
12 http://www.wta.org/go-hiking/hikes/tatoosh-lookout
13 http://www.wta.org/go-hiking/hikes/tatoosh
14 http://www.wta.org/go-hiking/hikes/mt.-margaret
15 http://www.wta.org/go-hiking/hikes/whittier-ridge-lakes-loop
16 http://www.wta.org/go-hiking/hikes/trap-lake
17 http://www.wta.org/go-hiking/hikes/red-mountain-lookout-1
18 http://www.wta.org/go-hiking/hikes/tradition-lakes
19 http://www.wt

In [113]:
hikes3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 37 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   name                                                       3412 non-null   object 
 1   url                                                        3412 non-null   object 
 2   length                                                     1921 non-null   float64
 3   highest point                                              1492 non-null   float64
 4   gain                                                       1492 non-null   float64
 5   lat                                                        2273 non-null   float64
 6   lon                                                        2273 non-null   float64
 7   pass: Discover Pass                                        3412 non-null   int64  
 8   pass: Na

In [115]:
len(hikes3[hikes3['lat'] != hikes3['lat2']])

1492

In [141]:
hikes3['lat'] = np.nanmean(np.concatenate((hikes3['lat'].values.reshape(-1,1),hikes3['lat2'].values.reshape(-1,1)), axis=1), axis=1)
hikes3['lon'] = np.nanmean(np.concatenate((hikes3['lon'].values.reshape(-1,1),hikes3['lon2'].values.reshape(-1,1)), axis=1), axis=1)
hikes3['gain'] = np.nanmean(np.concatenate((hikes3['gain'].values.reshape(-1,1),hikes3['gain2'].values.reshape(-1,1)), axis=1), axis=1)
hikes3['highest point'] = np.nanmean(np.concatenate((hikes3['highest point'].values.reshape(-1,1),hikes3['highest_point2'].values.reshape(-1,1)), axis=1), axis=1)
hikes3['length'] = np.nanmean(np.concatenate((hikes3['length'].values.reshape(-1,1),hikes3['length2'].values.reshape(-1,1)), axis=1), axis=1)
hikes3 = hikes3.drop(columns=['lat2','lon2','gain2','highest_point2','length2'])

In [142]:
hikes3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 32 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   name                                                       3412 non-null   object 
 1   url                                                        3412 non-null   object 
 2   length                                                     2193 non-null   float64
 3   highest point                                              1746 non-null   float64
 4   gain                                                       2043 non-null   float64
 5   lat                                                        2519 non-null   float64
 6   lon                                                        2519 non-null   float64
 7   pass: Discover Pass                                        3412 non-null   int64  
 8   pass: Na

In [143]:
hikes3.to_csv('wta-parks-data.csv', sep='\t')

In [144]:
hikes3.head()

,name,url,length,highest point,gain,lat,lon,pass: Discover Pass,pass: National Monument Fee,pass: National Park Pass,...,Old growth,Summits,Ridges/passes,Established campsites,Waterfalls,Rivers,rating,votes,reports,description
0,Luther Burbank Park,http://www.wta.org/go-hiking/hikes/luther-burb...,3.0,120.0,120.0,47.5890,-122.2270,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.00,5.0,15.0,\nLuther Burbank Park is a waterfront park at ...
1,Gem Lake,http://www.wta.org/go-hiking/hikes/gem-lake,10.0,4857.0,1800.0,47.4453,-121.4235,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.32,31.0,591.0,\nGem Lake is nestled at the foot of Wright Mo...
2,Talapus and Olallie Lakes,http://www.wta.org/go-hiking/hikes/talapus-lake,6.2,3780.0,1220.0,47.4007,-121.5190,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,3.87,101.0,1481.0,"\nWith a moderate grade on a wide trail, the h..."
3,Norse Peak,http://www.wta.org/go-hiking/hikes/norse-peak,10.0,6856.0,2900.0,46.9644,-121.4829,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.45,11.0,240.0,"\nWhile this trail is certainly steep, it's su..."
4,Rachel Lake,http://www.wta.org/go-hiking/hikes/rachel-lake,8.0,4600.0,1600.0,47.4012,-121.2848,0,0,0,...,1.0,0.0,1.0,1.0,1.0,0.0,4.01,68.0,956.0,\nFrom Rachel Lake you can access further adve...
